<a href="https://colab.research.google.com/github/Ankur3107/nlp_notebooks/blob/master/table-qa/TAPAS_fine_tuning_in_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

In [2]:
!pip install tensorflow_probability -qqq

In [3]:
import requests, zipfile, io
import os

def download_files(dir_name):
  if not os.path.exists(dir_name): 
    # 28 training examples from the SQA training set + table csv data
    urls = ["https://www.dropbox.com/s/2p6ez9xro357i63/sqa_train_set_28_examples.zip?dl=1",
            "https://www.dropbox.com/s/abhum8ssuow87h6/table_csv.zip?dl=1"
    ]
    for url in urls:
      r = requests.get(url)
      z = zipfile.ZipFile(io.BytesIO(r.content))
      z.extractall()

dir_name = "sqa_data"
download_files(dir_name)

In [4]:
import pandas as pd

data = pd.read_excel("sqa_train_set_28_examples.xlsx")
data.head()

,id,annotator,position,question,table_file,answer_coordinates,answer_text
0,nt-639,0,0,where are the players from?,table_csv/203_149.csv,"['(0, 4)', '(1, 4)', '(2, 4)', '(3, 4)', '(4, ...","['Louisiana State University', 'Valley HS (Las..."
1,nt-639,0,1,which player went to louisiana state university?,table_csv/203_149.csv,"['(0, 1)']",['Ben McDonald']
2,nt-639,1,0,who are the players?,table_csv/203_149.csv,"['(0, 1)', '(1, 1)', '(2, 1)', '(3, 1)', '(4, ...","['Ben McDonald', 'Tyler Houston', 'Roger Salke..."
3,nt-639,1,1,which ones are in the top 26 picks?,table_csv/203_149.csv,"['(0, 1)', '(1, 1)', '(2, 1)', '(3, 1)', '(4, ...","['Ben McDonald', 'Tyler Houston', 'Roger Salke..."
4,nt-639,1,2,"and of those, who is from louisiana state univ...",table_csv/203_149.csv,"['(0, 1)']",['Ben McDonald']


In [6]:
import ast

def _parse_answer_coordinates(answer_coordinate_str):
  """Parses the answer_coordinates of a question.
  Args:
    answer_coordinate_str: A string representation of a Python list of tuple
      strings.
      For example: "['(1, 4)','(1, 3)', ...]"
  """

  try:
    answer_coordinates = []
    # make a list of strings
    coords = ast.literal_eval(answer_coordinate_str)
    # parse each string as a tuple
    for row_index, column_index in sorted(
        ast.literal_eval(coord) for coord in coords):
      answer_coordinates.append((row_index, column_index))
  except SyntaxError:
    raise ValueError('Unable to evaluate %s' % answer_coordinate_str)
  
  return answer_coordinates


def _parse_answer_text(answer_text):
  """Populates the answer_texts field of `answer` by parsing `answer_text`.
  Args:
    answer_text: A string representation of a Python list of strings.
      For example: "[u'test', u'hello', ...]"
    answer: an Answer object.
  """
  try:
    answer = []
    for value in ast.literal_eval(answer_text):
      answer.append(value)
  except SyntaxError:
    raise ValueError('Unable to evaluate %s' % answer_text)

  return answer

data['answer_coordinates'] = data['answer_coordinates'].apply(lambda coords_str: _parse_answer_coordinates(coords_str))
data['answer_text'] = data['answer_text'].apply(lambda txt: _parse_answer_text(txt))

data.head(10)

,id,annotator,position,question,table_file,answer_coordinates,answer_text
0,nt-639,0,0,where are the players from?,table_csv/203_149.csv,"[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4...","[Louisiana State University, Valley HS (Las Ve..."
1,nt-639,0,1,which player went to louisiana state university?,table_csv/203_149.csv,"[(0, 1)]",[Ben McDonald]
2,nt-639,1,0,who are the players?,table_csv/203_149.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Ben McDonald, Tyler Houston, Roger Salkeld, J..."
3,nt-639,1,1,which ones are in the top 26 picks?,table_csv/203_149.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Ben McDonald, Tyler Houston, Roger Salkeld, J..."
4,nt-639,1,2,"and of those, who is from louisiana state univ...",table_csv/203_149.csv,"[(0, 1)]",[Ben McDonald]
5,nt-639,2,0,who are the players in the top 26?,table_csv/203_149.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Ben McDonald, Tyler Houston, Roger Salkeld, J..."
6,nt-639,2,1,"of those, which one was from louisiana state u...",table_csv/203_149.csv,"[(0, 1)]",[Ben McDonald]
7,nt-11649,0,0,what are all the names of the teams?,table_csv/204_135.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Cordoba CF, CD Malaga, Granada CF, UD Las Pal..."
8,nt-11649,0,1,"of these, which teams had any losses?",table_csv/204_135.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Cordoba CF, CD Malaga, Granada CF, UD Las Pal..."
9,nt-11649,0,2,"of these teams, which had more than 21 losses?",table_csv/204_135.csv,"[(15, 1)]",[CD Villarrobledo]


In [7]:
def get_sequence_id(example_id, annotator):
  if "-" in str(annotator):
    raise ValueError('"-" not allowed in annotator.')
  return f"{example_id}-{annotator}"

data['sequence_id'] = data.apply(lambda x: get_sequence_id(x.id, x.annotator), axis=1)
data.head()

,id,annotator,position,question,table_file,answer_coordinates,answer_text,sequence_id
0,nt-639,0,0,where are the players from?,table_csv/203_149.csv,"[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4...","[Louisiana State University, Valley HS (Las Ve...",nt-639-0
1,nt-639,0,1,which player went to louisiana state university?,table_csv/203_149.csv,"[(0, 1)]",[Ben McDonald],nt-639-0
2,nt-639,1,0,who are the players?,table_csv/203_149.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Ben McDonald, Tyler Houston, Roger Salkeld, J...",nt-639-1
3,nt-639,1,1,which ones are in the top 26 picks?,table_csv/203_149.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Ben McDonald, Tyler Houston, Roger Salkeld, J...",nt-639-1
4,nt-639,1,2,"and of those, who is from louisiana state univ...",table_csv/203_149.csv,"[(0, 1)]",[Ben McDonald],nt-639-1


In [8]:
# let's group table-question pairs by sequence id, and remove some columns we don't need 
grouped = data.groupby(by='sequence_id').agg(lambda x: x.tolist())
grouped = grouped.drop(columns=['id', 'annotator', 'position'])
grouped['table_file'] = grouped['table_file'].apply(lambda x: x[0])
grouped.head(10)

,question,table_file,answer_coordinates,answer_text
sequence_id,,,,
ns-1292-0,"[who are all the athletes?, where are they fro...",table_csv/204_521.csv,"[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, ...","[[Tommy Green, Janis Dalins, Ugo Frigerio, Kar..."
nt-10730-0,[what was the production numbers of each revol...,table_csv/203_253.csv,"[[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, ...","[[1,900 (estimated), 14,500 (estimated), 6,000..."
nt-10730-1,[what three revolver models had the least amou...,table_csv/203_253.csv,"[[(0, 0), (6, 0), (7, 0)], [(0, 0)]]","[[Remington-Beals Army Model Revolver, New Mod..."
nt-10730-2,"[what are all of the remington models?, how ma...",table_csv/203_253.csv,"[[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, ...","[[Remington-Beals Army Model Revolver, Remingt..."
nt-11649-0,"[what are all the names of the teams?, of thes...",table_csv/204_135.csv,"[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, ...","[[Cordoba CF, CD Malaga, Granada CF, UD Las Pa..."
nt-11649-1,"[what are the losses?, what team had more than...",table_csv/204_135.csv,"[[(0, 6), (1, 6), (2, 6), (3, 6), (4, 6), (5, ...","[[6, 6, 9, 10, 10, 12, 12, 11, 13, 14, 15, 14,..."
nt-11649-2,"[what were all the teams?, what were the loss ...",table_csv/204_135.csv,"[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, ...","[[Cordoba CF, CD Malaga, Granada CF, UD Las Pa..."
nt-639-0,"[where are the players from?, which player wen...",table_csv/203_149.csv,"[[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, ...","[[Louisiana State University, Valley HS (Las V..."
nt-639-1,"[who are the players?, which ones are in the t...",table_csv/203_149.csv,"[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, ...","[[Ben McDonald, Tyler Houston, Roger Salkeld, ..."


In [9]:
# path to the directory containing all csv files
table_csv_path = "table_csv"

item = grouped.iloc[0]
table = pd.read_csv(table_csv_path + item.table_file[9:]).astype(str) 

display(table)
print("")
print(item.question)

,Rank,Name,Nationality,Time (hand),Notes
0,nan,Tommy Green,Great Britain,4:50:10,OR
1,nan,Janis Dalins,Latvia,4:57:20,nan
2,nan,Ugo Frigerio,Italy,4:59:06,nan
3,4.0,Karl Hahnel,Germany,5:06:06,nan
4,5.0,Ettore Rivolta,Italy,5:07:39,nan
5,6.0,Paul Sievert,Germany,5:16:41,nan
6,7.0,Henri Quintric,France,5:27:25,nan
7,8.0,Ernie Crosbie,United States,5:28:02,nan
8,9.0,Bill Chisholm,United States,5:51:00,nan
9,10.0,Alfred Maasik,Estonia,6:19:00,nan



['who are all the athletes?', 'where are they from?', 'along with paul sievert, which athlete is from germany?']


In [10]:
import tensorflow as tf
from transformers import TapasTokenizer

# initialize the tokenizer
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base")

Downloading:   0%|          | 0.00/256k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/154 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/490 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

In [11]:
encoding = tokenizer(table=table, queries=item.question, answer_coordinates=item.answer_coordinates, answer_text=item.answer_text,
                     truncation=True, padding="max_length", return_tensors="tf")
encoding.keys()

dict_keys(['input_ids', 'labels', 'numeric_values', 'numeric_values_scale', 'token_type_ids', 'attention_mask'])

In [12]:
tokenizer.decode(encoding["input_ids"][0])

'[CLS] who are all the athletes? [SEP] rank name nationality time ( hand ) notes [EMPTY] tommy green great britain 4 : 50 : 10 or [EMPTY] janis dalins latvia 4 : 57 : 20 [EMPTY] [EMPTY] ugo frigerio italy 4 : 59 : 06 [EMPTY] 4. 0 karl hahnel germany 5 : 06 : 06 [EMPTY] 5. 0 ettore rivolta italy 5 : 07 : 39 [EMPTY] 6. 0 paul sievert germany 5 : 16 : 41 [EMPTY] 7. 0 henri quintric france 5 : 27 : 25 [EMPTY] 8. 0 ernie crosbie united states 5 : 28 : 02 [EMPTY] 9. 0 bill chisholm united states 5 : 51 : 00 [EMPTY] 10. 0 alfred maasik estonia 6 : 19 : 00 [EMPTY] [EMPTY] henry cieman canada [EMPTY] dnf [EMPTY] john moralis greece [EMPTY] dnf [EMPTY] francesco pretti italy [EMPTY] dnf [EMPTY] arthur tell schwab switzerland [EMPTY] dnf [EMPTY] harry hinkel united states [EMPTY] dnf [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [18]:
encoding["labels"][0]

<tf.Tensor: shape=(512,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     

In [19]:
class TableDataset:

    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer
    
    def __iter__(self):
        for idx in range(self.__len__()):
          item = self.df.iloc[idx]
          table = pd.read_csv(table_csv_path + item.table_file[9:]).astype(str) # TapasTokenizer expects the table data to be text only
          if item.position != 0:
            # use the previous table-question pair to correctly set the prev_labels token type ids
            previous_item = self.df.iloc[idx-1]
            encoding = self.tokenizer(table=table, 
                                      queries=[previous_item.question, item.question], 
                                      answer_coordinates=[previous_item.answer_coordinates, item.answer_coordinates], 
                                      answer_text=[previous_item.answer_text, item.answer_text],
                                      padding="max_length",
                                      truncation=True,
                                      return_tensors="tf"
            )
            # use encodings of second table-question pair in the batch
            encoding = {key: val[-1] for key, val in encoding.items()}
          else:
            # this means it's the first table-question pair in a sequence
            encoding = self.tokenizer(table=table, 
                                      queries=item.question, 
                                      answer_coordinates=item.answer_coordinates, 
                                      answer_text=item.answer_text,
                                      padding="max_length",
                                      truncation=True,
                                      return_tensors="tf"
            )
            # remove the batch dimension which the tokenizer adds 
            encoding = {key: tf.squeeze(val,0) for key, val in encoding.items()}
          yield encoding['input_ids'],encoding['attention_mask'],encoding['numeric_values'], \
                encoding['numeric_values_scale'], encoding['token_type_ids'], encoding['labels']

    def __len__(self):
        return len(self.df)
    
    __call__ = __iter__

train_dataset = TableDataset(df=data, tokenizer=tokenizer)
output_signature = (
         tf.TensorSpec(shape=(512,), dtype=tf.int32),
         tf.TensorSpec(shape=(512,), dtype=tf.int32),
         tf.TensorSpec(shape=(512,), dtype=tf.float32),
         tf.TensorSpec(shape=(512,), dtype=tf.float32),
         tf.TensorSpec(shape=(512,7), dtype=tf.int32),
         tf.TensorSpec(shape=(512,), dtype=tf.int32) )
train_dataloader = tf.data.Dataset.from_generator(train_dataset,output_signature=output_signature).batch(2)

In [20]:
batch = next(iter(train_dataloader))

In [21]:
batch[0].shape

TensorShape([2, 512])

In [22]:
batch[4].shape

TensorShape([2, 512, 7])

In [24]:
batch[2].shape

TensorShape([2, 512])

In [27]:
tokenizer.decode(batch[0][1])

'[CLS] which player went to louisiana state university? [SEP] pick player team position school 1 ben mcdonald baltimore orioles rhp louisiana state university 2 tyler houston atlanta braves c valley hs ( las vegas, nv ) 3 roger salkeld seattle mariners rhp saugus ( ca ) hs 4 jeff jackson philadelphia phillies of simeon hs ( chicago, il ) 5 donald harris texas rangers of texas tech university 6 paul coleman saint louis cardinals of frankston ( tx ) hs 7 frank thomas chicago white sox 1b auburn university 8 earl cunningham chicago cubs of lancaster ( sc ) hs 9 kyle abbott california angels lhp long beach state university 10 charles johnson montreal expos c westwood hs ( fort pierce, fl ) 11 calvin murray cleveland indians 3b w. t. white high school ( dallas, tx ) 12 jeff juden houston astros rhp salem ( ma ) hs 13 brent mayne kansas city royals c cal state fullerton 14 steve hosey san francisco giants of fresno state university 15 kiki jones los angeles dodgers rhp hillsborough hs ( tamp

In [28]:
assert sum(batch[4][0][:,3]) == 0

In [29]:
print(sum(batch[4][1][:,3]))

tf.Tensor(132, shape=(), dtype=int32)


In [30]:
for id, prev_label in zip(batch[0][1], batch[4][1][:,3]):
  if id != 0:
    print(tokenizer.decode([id]), prev_label.numpy().item())

[CLS] 0
which 0
player 0
went 0
to 0
louisiana 0
state 0
university 0
? 0
[SEP] 0
pick 0
player 0
team 0
position 0
school 0
1 0
ben 0
mcdonald 0
baltimore 0
orioles 0
r 0
##hp 0
louisiana 1
state 1
university 1
2 0
tyler 0
houston 0
atlanta 0
braves 0
c 0
valley 1
hs 1
( 1
las 1
vegas 1
, 1
n 1
##v 1
) 1
3 0
roger 0
sal 0
##kel 0
##d 0
seattle 0
mariners 0
r 0
##hp 0
sa 1
##ug 1
##us 1
( 1
ca 1
) 1
hs 1
4 0
jeff 0
jackson 0
philadelphia 0
phillies 0
of 0
simeon 1
hs 1
( 1
chicago 1
, 1
il 1
) 1
5 0
donald 0
harris 0
texas 0
rangers 0
of 0
texas 1
tech 1
university 1
6 0
paul 0
coleman 0
saint 0
louis 0
cardinals 0
of 0
franks 1
##ton 1
( 1
tx 1
) 1
hs 1
7 0
frank 0
thomas 0
chicago 0
white 0
sox 0
1b 0
auburn 1
university 1
8 0
earl 0
cunningham 0
chicago 0
cubs 0
of 0
lancaster 1
( 1
sc 1
) 1
hs 1
9 0
kyle 0
abbott 0
california 0
angels 0
l 0
##hp 0
long 1
beach 1
state 1
university 1
10 0
charles 0
johnson 0
montreal 0
expo 0
##s 0
c 0
westwood 1
hs 1
( 1
fort 1
pierce 1
, 1
fl 1
) 

In [31]:
from transformers import TFTapasForQuestionAnswering

model = TFTapasForQuestionAnswering.from_pretrained("google/tapas-base")

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFTapasForQuestionAnswering.

Some layers of TFTapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['compute_column_logits', 'dropout_37', 'compute_token_logits']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
model.config.num_aggregation_labels

0

In [33]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)


for epoch in range(10):  # loop over the dataset multiple times
   print("Epoch:", epoch)
   for idx, batch in enumerate(train_dataloader):
        # get the inputs;
        input_ids = batch[0]
        attention_mask = batch[1]
        token_type_ids = batch[4]
        labels = batch[-1]
        
        with tf.GradientTape() as tape:
          outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,
                        labels=labels, training=True)
        
        print("loss: ",outputs.loss.numpy().item())
        grads = tape.gradient(outputs.loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

Epoch: 0
loss:  2.2713990211486816
loss:  2.0187385082244873
loss:  1.3549939393997192


KeyboardInterrupt: ignored

In [34]:
import tensorflow_probability as tfp

In [35]:
import collections
import numpy as np

def compute_prediction_sequence(model, data):
  """Computes predictions using model's answers to the previous questions."""
  
  # prepare data
  all_logits = []
  prev_answers = None
  batch_size = inputs["input_ids"].shape[0]

  input_ids = inputs["input_ids"]
  attention_mask = inputs["attention_mask"]
  token_type_ids = inputs["token_type_ids"]
  token_type_ids_example = None

  for index in range(batch_size):
      # If sequences have already been processed, the token type IDs will be created according to the previous
      # answer.
      if prev_answers is not None:
          prev_labels_example = token_type_ids_example[:, 3]  # shape (seq_len,)
          model_labels = np.zeros_like(prev_labels_example, dtype=np.int32)  # shape (seq_len,)

          token_type_ids_example = token_type_ids[index].numpy()  # shape (seq_len, 7)
          for i in range(model_labels.shape[0]):
              segment_id = token_type_ids_example[:, 0].tolist()[i]
              col_id = token_type_ids_example[:, 1].tolist()[i] - 1
              row_id = token_type_ids_example[:, 2].tolist()[i] - 1

              if row_id >= 0 and col_id >= 0 and segment_id == 1:
                  model_labels[i] = int(prev_answers[(col_id, row_id)])

          token_type_ids_example[:, 3] = model_labels

      input_ids_example = input_ids[index]
      attention_mask_example = attention_mask[index]  # shape (seq_len,)
      token_type_ids_example = token_type_ids[index]  # shape (seq_len, 7)
      outputs = model(
          input_ids=np.expand_dims(input_ids_example, axis=0),
          attention_mask=np.expand_dims(attention_mask_example, axis=0),
          token_type_ids=np.expand_dims(token_type_ids_example, axis=0),
      )
      logits = outputs.logits


      all_logits.append(logits)

      dist_per_token = tfp.distributions.Bernoulli(logits=logits)
      probabilities = dist_per_token.probs_parameter() * tf.cast(attention_mask_example, tf.float32)

      coords_to_probs = collections.defaultdict(list)
      token_type_ids_example = token_type_ids_example.numpy()
      for i, p in enumerate(tf.squeeze(probabilities).numpy().tolist()):
          segment_id = token_type_ids_example[:, 0].tolist()[i]
          col = token_type_ids_example[:, 1].tolist()[i] - 1
          row = token_type_ids_example[:, 2].tolist()[i] - 1
          if col >= 0 and row >= 0 and segment_id == 1:
              coords_to_probs[(col, row)].append(p)

      prev_answers = {key: np.array(coords_to_probs[key]).mean() > 0.5 for key in coords_to_probs}

  logits_batch = tf.concat(tuple(all_logits), 0)

  return logits_batch

In [36]:
data = {'Actors': ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], 
        'Age': ["56", "45", "59"],
        'Number of movies': ["87", "53", "69"],
        'Date of birth': ["7 february 1967", "10 june 1996", "28 november 1967"]}
queries = ["How many movies has George Clooney played in?", "How old is he?", "What's his date of birth?"]

table = pd.DataFrame.from_dict(data)

inputs = tokenizer(table=table, queries=queries, padding='max_length', return_tensors="tf")
logits = compute_prediction_sequence(model, inputs)

In [37]:
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits)

In [38]:
predicted_answer_coordinates

[[(0, 3), (1, 3), (2, 3)], [(0, 3), (1, 3), (2, 3)], [(0, 3), (1, 3), (2, 3)]]

In [39]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(table.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(table.iat[coordinate])
    answers.append(", ".join(cell_values))

display(table)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

,Actors,Age,Number of movies,Date of birth
0,Brad Pitt,56,87,7 february 1967
1,Leonardo Di Caprio,45,53,10 june 1996
2,George Clooney,59,69,28 november 1967



How many movies has George Clooney played in?
Predicted answer: 7 february 1967, 10 june 1996, 28 november 1967
How old is he?
Predicted answer: 7 february 1967, 10 june 1996, 28 november 1967
What's his date of birth?
Predicted answer: 7 february 1967, 10 june 1996, 28 november 1967


# Reference 
https://github.com/kamalkraj/Tapas-Tutorial
